In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, models, transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [ ]:
# Khai báo đường dẫn đến dataset đã được tăng cường trước đó
data_dir = r"C:\Users\nguye\Downloads\ML-DL\Dataset\AlzheimerData_Augmented"
# Định nghĩa các phép biến đổi dữ liệu
# Chúng giúp chuẩn hóa và tiền xử lý ảnh để phù hợp với mô hình
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val_test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [3]:
# Tải dữ liệu bằng ImageFolder
full_dataset = datasets.ImageFolder(data_dir, data_transforms['val_test']) 
class_names = full_dataset.classes
print(f"Các lớp tìm thấy: {class_names}")

Các lớp tìm thấy: ['Mild Dementia', 'Moderate Dementia', 'Non Demented', 'Very mild Dementia']


In [4]:
# Chia dataset thành các tập train, validation và test
dataset_size = len(full_dataset)
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = dataset_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

# Áp dụng transform huấn luyện cho tập train
train_dataset.dataset.transform = data_transforms['train']

print(f"Tổng số mẫu: {dataset_size}")
print(f"Kích thước tập huấn luyện: {len(train_dataset)}")
print(f"Kích thước tập kiểm tra: {len(val_dataset)}")
print(f"Kích thước tập đánh giá: {len(test_dataset)}")

Tổng số mẫu: 268217
Kích thước tập huấn luyện: 187751
Kích thước tập kiểm tra: 40232
Kích thước tập đánh giá: 40234


In [5]:
# Tạo DataLoader để load dữ liệu theo lô (batch)
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [6]:
# Tải mô hình ResNet-101 đã được huấn luyện trước
model = models.resnet101(pretrained=True)

C:\Users\nguye\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\nguye\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# Đóng băng các tham số của mô hình
for param in model.parameters():
    param.requires_grad = False

In [8]:
# Thay thế lớp cuối cùng để phù hợp với số lượng lớp của bạn
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

In [9]:
# Chuyển đổi mô hình sang GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Sử dụng thiết bị: {device}")

Sử dụng thiết bị: cuda


In [10]:
# Định nghĩa hàm mất mát và bộ tối ưu
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
# Bắt đầu quá trình huấn luyện Giai đoạn 1
epochs_phase1 = 5
print(f"\n--- Bắt đầu Giai đoạn 1: Huấn luyện lớp cuối cùng ({epochs_phase1} epochs) ---")
best_val_acc = 0.0

for epoch in range(epochs_phase1):
    print(f'Epoch {epoch+1}/{epochs_phase1}')
    print('-' * 10)

    model.train()
    for inputs, labels in tqdm(train_loader, desc="Training Phase 1"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # Kiểm tra hiệu suất trên tập validation sau mỗi epoch
    model.eval()
    running_corrects = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
    
    val_acc = running_corrects.double() / len(val_dataset)
    print(f'Độ chính xác kiểm tra Giai đoạn 1: {val_acc:.4f}')

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model_phase1.pth')
        print("Đã lưu mô hình tốt nhất Giai đoạn 1!")


--- Bắt đầu Giai đoạn 1: Huấn luyện lớp cuối cùng (5 epochs) ---
Epoch 1/5
----------


Training Phase 1: 100%|████████████████████████████████████████████████████████████| 5868/5868 [25:21<00:00,  3.86it/s]


Độ chính xác kiểm tra Giai đoạn 1: 0.7233
Đã lưu mô hình tốt nhất Giai đoạn 1!
Epoch 2/5
----------


Training Phase 1: 100%|████████████████████████████████████████████████████████████| 5868/5868 [25:55<00:00,  3.77it/s]


Độ chính xác kiểm tra Giai đoạn 1: 0.6985
Epoch 3/5
----------


...

Độ chính xác kiểm tra Giai đoạn 1: 0.7414
Đã lưu mô hình tốt nhất Giai đoạn 1!
Epoch 4/5
----------


Training Phase 1: 100%|████████████████████████████████████████████████████████████| 5868/5868 [25:46<00:00,  3.79it/s]


Độ chính xác kiểm tra Giai đoạn 1: 0.7484
Đã lưu mô hình tốt nhất Giai đoạn 1!
Epoch 5/5
----------


Training Phase 1: 100%|████████████████████████████████████████████████████████████| 5868/5868 [25:50<00:00,  3.78it/s]


Độ chính xác kiểm tra Giai đoạn 1: 0.7496
Đã lưu mô hình tốt nhất Giai đoạn 1!


In [12]:
# Giai đoạn 2: Tinh chỉnh (fine-tuning) toàn bộ mô hình
# Mở đóng băng tất cả các tham số
for param in model.parameters():
    param.requires_grad = True

# Định nghĩa lại bộ tối ưu với tốc độ học thấp hơn cho toàn bộ mô hình
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Tải lại mô hình tốt nhất từ giai đoạn 1
model.load_state_dict(torch.load('best_model_phase1.pth'))

epochs_phase2 = 10
print(f"\n--- Bắt đầu Giai đoạn 2: Tinh chỉnh toàn bộ mô hình ({epochs_phase2} epochs) ---")
best_val_acc = 0.0

for epoch in range(epochs_phase2):
    print(f'Epoch {epoch+1}/{epochs_phase2}')
    print('-' * 10)
    
    model.train()
    for inputs, labels in tqdm(train_loader, desc="Training Phase 2"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
    model.eval()
    running_corrects = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
    
    val_acc = running_corrects.double() / len(val_dataset)
    print(f'Độ chính xác kiểm tra Giai đoạn 2: {val_acc:.4f}')

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'final_best_model.pth')
        print("Đã lưu mô hình tốt nhất Giai đoạn 2!")

print("\n--- Huấn luyện hoàn tất ---")
print(f"Độ chính xác cao nhất trên tập kiểm tra: {best_val_acc:.4f}")


--- Bắt đầu Giai đoạn 2: Tinh chỉnh toàn bộ mô hình (10 epochs) ---
Epoch 1/10
----------


Training Phase 2:   1%|▌                                                          | 51/5868 [06:00<11:25:47,  7.07s/it]


KeyboardInterrupt: 

In [ ]:
# Đánh giá mô hình trên tập test
model.load_state_dict(torch.load('final_best_model.pth'))
model.eval()
all_preds = []
all_labels = []

print("\n--- Đánh giá mô hình trên tập test ---")
for inputs, labels in tqdm(test_loader, desc="Testing"):
    inputs, labels = inputs.to(device), labels.to(device)
    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("\nBáo cáo phân loại:")
print(classification_report(all_labels, all_preds, target_names=class_names))

print("Ma trận nhầm lẫn:")
conf_matrix = confusion_matrix(all_labels, all_preds)
print(conf_matrix)